In [21]:
from mnist import main
from kubeflow.training import TrainingClient

In [22]:
job = TrainingClient().create_job(
    name="pytorch-ddp",
    train_func=main,
    num_procs_per_worker="auto",
    num_workers=3,
    parameters={},
    base_image="quay.io/rhoai/ray:2.35.0-py311-cu121-torch24-fa26",
    packages_to_install=[  # NOTE extra index doesn't seem to be possible for ROCm packages from what I can tell only a single index can be specified
                           # see here: https://github.com/kubeflow/training-operator/blob/6a898861d9a85266240ecfefc6574fe5fc7c3063/sdk/python/kubeflow/training/api/training_client.py#L338
                           # There is the environment variable PIP_EXTRA_INDEX_URL which can be used instead. Adding this functionality to kfto-sdk might be a good idea
        "pytorch_lightning==1.9.5",
        "ray_lightning",
        "torchmetrics==0.9.1",
        "torchvision==0.19.0",
        "minio",
    ]
)

/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
TrainingClient().get_job_logs("pytorch-ddp")

/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


({'pytorch-ddp-master-0': "WARNING: The directory '/.cache/pip' or its parent directory is not owned or is not writable by the current user. The cache has been disabled. Check the permissions and owner of that directory. If executing pip with sudo, you should use sudo's -H flag.\nERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/.local'\nCheck the permissions.\n\nbash: line 8: nccl: command not found\n"},
 {})

In [25]:
TrainingClient().delete_job(name="pytorch-ddp")

/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
